# Just How Random These Candy Codes Are

Received a few of golden candy papers with codes in them and started to wonder the randomness of the codes.

![Golden Candy Paper](candypaper.jpg)

The source of the data is candy papers collected from bags of candies.

This is good excuse to study how randomness is tested.


## Source Data

To add bit more data into this a bunch of data samples were created with different sources and methods:

1. `/dev/urandom` provided by Linux kernel https://linux.die.net/man/4/urandom 
2. Hotbits provided by FourmiLabs https://www.fourmilab.ch/hotbits/
3. `random.org` service https://www.random.org/
4. Posix Pseudo Random Number Generator `rand()` https://linux.die.net/man/3/rand
5. Python `random` library https://docs.python.org/3.6/library/random.html
6. Middle Square algorithm https://en.wikipedia.org/wiki/Middle-square_method

Data was created as 128 bytes and 128 character string with upper case (A-Z) letters. 

## Preparation

List of preparations done for processing the data

In [9]:
import csv


def read_csv(file):
    data = []
    with open(file, newline='') as csvfile:
        reader = csv.reader(csvfile)
        header = True
        for row in reader:
            if not header:
                data.append(row[1].strip())
            else:
                # ignore header/first row
                header = False
    return ''.join(data)


def read_bytes(file):
    with open(file, 'rb') as f:
        return bytes(f.read())
    

def read_string(file):
    with open(file, 'r') as f:
        return str(f.read())
    

config = {
    'data.csv': {'text': True, 'reader': read_csv},
    'hotbits.bin': {'text': False, 'reader': read_bytes},
    'hotbits.txt': {'text': True, 'reader': read_string},
    'middlesquare.bin': {'text': False, 'reader': read_bytes},
    'middlesquare.txt': {'text': True, 'reader': read_string},
    'pyrandom.bin': {'text': False, 'reader': read_bytes},
    'pyrandom.txt': {'text': True, 'reader': read_string},
    'rand.bin': {'text': False, 'reader': read_bytes},
    'rand.txt': {'text': True, 'reader': read_string},
    'random.org.bin': {'text': False, 'reader': read_bytes},
    'random.org.txt': {'text': True, 'reader': read_string},
    'urandom.bin': {'text': False, 'reader': read_bytes},
    'urandom.txt': {'text': True, 'reader': read_string}    
}

## Import the Data

Import the data from files

In [10]:
def read_data(config):
    data = {}
    for key in config.keys():
        data[key] = config[key]['reader'](key)
    return data


data = read_data(config)

## Average Testing

Simple test is to calculate the average of the values. Random values should be evenly distributed and the average should be in the middle of the range. In case random bytes it should be `127.5` and for Upper case ASCII strings the average should be `77.5`.

In [31]:
def print_average(name, value):
    print(f"{name:16} {value:5.1f}")


def average_bytes(config):
    def average(test_data):
        return sum(test_data)/len(test_data)
    for key in config.keys():
        if not config[key]['text']:
            print_average(key, average(data[key]))


def average_chars(config):
    def average(test_data):
        return sum([ord(x) for x in test_data])/len(test_data)
    for key in config.keys():
        if config[key]['text']:
            print_average(key, average(data[key]))

For random byte data the average values are:

In [32]:
average_bytes(config)

hotbits.bin      119.6
middlesquare.bin 115.9
pyrandom.bin     128.6
rand.bin         134.8
random.org.bin   123.3
urandom.bin      126.7


For random upper case character data the average values are:

In [33]:
average_chars(config)

data.csv          75.9
hotbits.txt       78.5
middlesquare.txt  75.9
pyrandom.txt      78.0
rand.txt          76.7
random.org.txt    75.9
urandom.txt       77.6


From the average we can't really say that these values are not random. The data sets are small and values are close.